In [1]:
from nsetools import Nse
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import datetime as dt
from datetime import datetime
import pickle5 as pickle
import threading
import os
import json
import time

In [2]:
url = "https://www1.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp"
curtime = datetime.now().strftime("%H:%M:%S")

#Create dataframe
strikes = [50*i for i in range(220,281)]
df=pd.DataFrame(columns = ['strike_price'], index = strikes)
for i in strikes:
    df.loc[i,'strike_price'] = i

In [3]:
def get_soup(url):
    response = requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
    if(response.status_code != 200):
        print('Illegal URL')
        return None
    soup = BeautifulSoup(response.text,'lxml')
    print("Soup generation. 200!")
    return soup

In [4]:
def get_nifty(soup):
    try:
        val1 = soup.find_all('span')[0].text.split()[-1]
        val2 = soup.find_all('span')[1].text.split()[-2]
        return ({'nifty':val1,'time':val2})
    except:
        print("Bad request")
        return None

In [5]:
def insert_values(soup, nifty, dataframe):
    cols = [
        'CE_OI_@'+nifty['time'],
        'CE_CHOI_@'+nifty['time'],
        'CE_IV_@'+nifty['time'],
        'CE_LTP_@'+nifty['time'],
        'PE_OI_@'+nifty['time'],
        'PE_CHOI_@'+nifty['time'],
        'PE_IV_@'+nifty['time'],
        'PE_LTP_@'+nifty['time'],
        'NIFTY@'+nifty['time'],
        'CE_VOL_@'+nifty['time'],
        'PE_VOL_@'+nifty['time'],
        'CE_LTP_CHG_@'+nifty['time'],
        'PE_LTP_CHG_@'+nifty['time']
    ]
    
    for col in cols:
        dataframe[col] = np.nan
    
    for one in soup.find_all('div',{'class':'opttbldata'})[0].find_all('tr')[2:-1]:
        one = one.find_all('td')
        strike = one[11].text
        if(int(float(strike)) not in list(df['strike_price'])):
            continue
        values = dict()
        values[cols[0]] = one[1].text
        values[cols[1]] = one[2].text
        values[cols[2]] = one[4].text
        values[cols[3]] = one[5].text
        values[cols[4]] = one[21].text
        values[cols[5]] = one[20].text
        values[cols[6]] = one[18].text
        values[cols[7]] = one[17].text
        values[cols[8]] = nifty['nifty']
        values[cols[9]] = one[3].text
        values[cols[10]] = one[19].text
        values[cols[11]] = one[6].text
        values[cols[12]] = one[16].text
        
    
        for key in values:
            values[key] = values[key].replace(" ","")
            values[key] = values[key].replace("\t","")
            values[key] = values[key].replace("\n","")
            values[key] = values[key].replace("\r","")
            values[key] = values[key].replace("\n","")
            values[key] = values[key].replace(",","")
            try: 
                values[key] = float(values[key])
            except:
                values[key] = np.nan
            
            dataframe.loc[int(float(strike)),key] = values[key]

    return dataframe

In [6]:
def automation(curtime, df):
    
    soup = get_soup(url)
    nifty = get_nifty(soup)
    print(nifty)
    
    if(curtime == nifty['time']):
        print("Failed to add! time:"+curtime)
        return curtime,df
    else:
        cutime = nifty['time']
        
    df=insert_values(soup, nifty, df)
    df.to_csv('data.csv',index=True)
    
    return nifty['time'],df

In [7]:
#curtime,df = automation(curtime, df)

In [8]:
while True:
    print("Start")
    curtime,df = automation(curtime, df)
    time.sleep(60*7)

Start
Soup generation. 200!
{'nifty': '13170.35', 'time': '09:17:20'}
Start
Soup generation. 200!
{'nifty': '13163.80', 'time': '09:22:50'}
Start
Soup generation. 200!
{'nifty': '13162.05', 'time': '09:30:50'}
Start
Soup generation. 200!
{'nifty': '13155.25', 'time': '09:37:20'}
Start
Soup generation. 200!
{'nifty': '13154.60', 'time': '09:44:50'}
Start
Soup generation. 200!
{'nifty': '13161.30', 'time': '09:51:50'}
Start
Soup generation. 200!
{'nifty': '13162.05', 'time': '09:59:20'}
Start
Soup generation. 200!
{'nifty': '13153.05', 'time': '10:06:21'}
Start
Soup generation. 200!
{'nifty': '13153.65', 'time': '10:12:21'}
Start
Soup generation. 200!
{'nifty': '13164.85', 'time': '10:19:51'}
Start
Soup generation. 200!
{'nifty': '13178.65', 'time': '10:26:51'}
Start
Soup generation. 200!
{'nifty': '13173.55', 'time': '10:33:51'}
Start
Soup generation. 200!
{'nifty': '13162.60', 'time': '10:40:51'}
Start
Soup generation. 200!
{'nifty': '13160.65', 'time': '10:47:52'}
Start
Soup generatio

KeyboardInterrupt: 